# Bitcoin Key Pairs

In [1]:
# pip install ecdsa

## Manual Key Generation

In [2]:
import ecdsa
import binascii


# # Generate a private key
private_key_raw = ecdsa.SigningKey.generate(curve=ecdsa.SECP256k1)
print("Private key raw:", private_key_raw.to_string().hex())

# Get the private key
# private_key_hex = private_key_raw.to_string().hex()
# private_key_bytes = binascii.unhexlify(private_key_hex)
# private_key = ecdsa.SigningKey.from_string(private_key_bytes, curve=ecdsa.SECP256k1)

# Get the public key
public_key: ecdsa.VerifyingKey = private_key_raw.get_verifying_key()

# Print the public key in hexadecimal format
print(f"Public key: {public_key.to_string().hex()}")


def compress_public_key(public_key: bytes) -> bytes:
    x = public_key[:32]
    y = public_key[32:]
    if int.from_bytes(y, 'big') % 2 == 0:
        return b'\x02' + x
    else:
        return b'\x03' + x
    
# Print the compressed public key
print(f"Compressed public key: {compress_public_key(public_key.to_string()).hex()}")



# # Convert private key to hex
# private_key = private_key_raw.to_string().hex() + '01'
# print("Private key:", private_key)

# # Generate a public key
# public_key = private_key_raw.get_verifying_key()
# print("Public key: ", public_key.to_string().hex())

Private key raw: 3839cf91e611d1b8f743ad1aa3bf9a6673752e95c1e2fd4d07bca30cbc2f9bcf
Public key: 576a807151ae7bf4c279560a50f3edc2796f2727784cf250b39221688cf61966e9e08ab7d495fe5f0795d84708800262397b23acec4596a61c0285c48b731e45
Compressed public key: 03576a807151ae7bf4c279560a50f3edc2796f2727784cf250b39221688cf61966


While the public key is correct it is not in the proper format. 

## Bitcoin Address

Finding the bitcoin public key or bitcoin address from the private key is a bit more complicated.

The public key is a point on the elliptic curve, but it is not in the proper format. The proper format is a 65 byte array with the first byte being 0x04 and the next 32 bytes being the x coordinate and the last 32 bytes being the y coordinate. The x and y coordinates are 32 bytes each because they are 256 bit numbers. The first byte is 0x04 because it is an uncompressed public key. There are compressed public keys which are 33 bytes long, but we will not be using them.

### Create a bitcoin address

In [3]:
import hashlib
import base58

# Generate a bitcoin address
# Step 1: Hash public key
sha256: bytes = hashlib.sha256(public_key.to_string().hex().encode()).digest()

# Step 2: Perform RIPEMD-160 hashing on the result of SHA256
ripemd160 = hashlib.new('ripemd160')
ripemd160.update(sha256)
hashed_public_key = ripemd160.digest()

# Step 3: Add version byte in front of RIPEMD-160 hash (0x00 for Main Network)
hashed_public_key = b'\x00' + hashed_public_key

# Step 4: Perform SHA256 hash on the extended RIPEMD-160 result
double_sha256 = hashlib.sha256(hashlib.sha256(hashed_public_key).digest()).digest()

# Step 5: Take the first 4 bytes of the second SHA256 hash. This is the address checksum
address_checksum = double_sha256[:4]

# Step 6: Add the 4 checksum bytes from stage 5 at the end of extended RIPEMD-160 hash from stage 4. This is the 25-byte binary Bitcoin Address.
binary_bitcoin_address = hashed_public_key + address_checksum

# Step 7: Convert the result from a byte string into a base58 string using Base58Check encoding. This is the most commonly used Bitcoin Address format
bitcoin_address = base58.b58encode(binary_bitcoin_address).decode("ascii")
print("Bitcoin address: ", bitcoin_address)

Bitcoin address:  196fqHTSGuyHJYJH7MMs2GP49cd8ZHc1yG


### Verify a bitcoin address

In [4]:
# Verify that the bitcoin address is correct
# Step 1: Decode Base58 Bitcoin Address
binary_bitcoin_address_check = base58.b58decode(bitcoin_address)

# Step 2: Extract Bitcoin Address from decoded Base58 Bitcoin Address
binary_bitcoin_address = binary_bitcoin_address_check[:-4]

# Step 3: Extract Bitcoin Address Checksum from decoded Base58 Bitcoin Address
binary_bitcoin_address_checksum = binary_bitcoin_address_check[-4:]

# Step 4: Perform SHA256 hash on the extended RIPEMD-160 result
double_sha256 = hashlib.sha256(hashlib.sha256(binary_bitcoin_address).digest()).digest()

# Step 5: Take the first 4 bytes of the second SHA256 hash. This is the address checksum
address_checksum = double_sha256[:4]

# Step 6: Verify that the address checksum from decoded Base58 Bitcoin Address is equal to the address checksum from the SHA256 hash
print("Bitcoin address is valid:", binary_bitcoin_address_checksum == address_checksum)

Bitcoin address is valid: True


## Signing Messages

### Sign a message

In [5]:
# Path: concepts_bitcoin_transactions.ipynb

# Use the private key to sign a message
# message = bytes("Bitcoin Signed Message:\nHello world!", "utf-8")
message = b"\x18Bitcoin Signed Message:\n" + b"Hello world!"
signature = private_key_raw.sign_deterministic(message)
# print("Signature:", signature.__bytes__().hex())
print("Signature:", signature.hex())


Signature: b42cf72536589b6a8ed998f9a32d8a2cc7c77e24801b223a4368bfee8097b386350721d994c67bd0b1cbff8ed4f48e74cd19c974fb9329760c5cad1f726c17d3


### Verify a message

In [6]:
# Use the public key to verify a message
print(public_key.verify(signature, message))

True


# Libraries for bitcoin keys

### pybitcoin

No longer maintained and does not work with python 3.12

In [7]:
# pip install pybitcoin

In [8]:
# from pybitcoin import BitcoinPrivateKey

# # Generate a private key
# private_key = BitcoinPrivateKey()
# print(f"Private key: {private_key.to_hex()}")

# # Generate a public key
# public_key = private_key.public_key()
# print(f"Public key: {public_key.to_hex()}")

# # Generate a bitcoin address
# bitcoin_address = public_key.address()
# print(f"Bitcoin address: {bitcoin_address}")

### bitcoin-utils

Package appears damaged and does not work with python 3.12

In [9]:
# pip install bitcoin-utils

In [10]:
# from bitcoinutils.keys import PrivateKey, PublicKey

# # Generate a private key
# private_key = PrivateKey()
# print(f"Private key: {private_key.to_wif(compressed=True)}")

# # get the public key
# public_key: PublicKey = private_key.get_public_key()
# print(f"Public key: {public_key.to_hex(compressed=True)}")

# # get the bitcoin address
# bitcoin_address = public_key.get_address()
# print(f"Bitcoin address: {bitcoin_address.to_string()}")
# print(f'Hash160: {bitcoin_address.to_hash160()}')



Sign a message with a private key and verify the signature with the public key.

In [11]:
# message: str = "Hello"
# signature = private_key.sign_message(message)
# print(f"Signature: {signature}")
# # print(f"Signature valid? {public_key.verify(message, signature)}")
# print(f"Signature valid? {public_key.verify_message(public_key.get_address().to_string(), message, signature)}")

### python-bitcoinlib

In [12]:
# pip install python-bitcoinlib

In [13]:
import secrets
import base64
from bitcoin.wallet import CBitcoinSecret, P2PKHBitcoinAddress, CBitcoinAddress
from bitcoin.signmessage import BitcoinMessage, VerifyMessage, SignMessage
from bitcoin.core import x, lx, b2x, b2lx, Hash160

message_: str = "Hello world!"

# Generate a private key
private_key_ = CBitcoinSecret.from_secret_bytes(secrets.token_bytes(32))
print(f"Private key: {private_key_}")

# Generate a public key
public_key_ = private_key_.pub
print(f"Public key: {public_key_.hex()}")

# Generate a bitcoin address
bitcoin_address_ = P2PKHBitcoinAddress.from_pubkey(public_key_)
# bitcoin_address_ = CBitcoinAddress(bitcoin_address_)
print(f"Bitcoin address: {bitcoin_address_}")

# Sign message
bitcoin_msg = BitcoinMessage(message_)
print(f"Bitcoin message: {bitcoin_msg.message}")

signature_ = SignMessage(private_key_, bitcoin_msg)
print(f"Signature: {signature_.hex()}")

# verify the message
print(f"Signature valid? {VerifyMessage(bitcoin_address_, bitcoin_msg, signature_)}")


Private key: L59Ho2m9qGE7cvhb2V9MqJnrtZYTCWMPNknu6QNWG6G5Vwc31wT3
Public key: 0347bd324977e2ae469fdc9e8d03002d6689c600208e47dc7b94af69924808d599
Bitcoin address: 1FU2yKCnXYum2jsBJHGNgJA3pXYYyxpdk8
Bitcoin message: b'Hello world!'
Signature: 483236663931345865596e57455031744c665a5a6f6a2f597a74424a52317769594739316f79705479517931615155552b7751752b6368442b594b67504a6358447141714c3235424a4b79504d4e4470786135754c2b593d
Signature valid? True


In [14]:
# verify the handmade message from the top
# bitcoin_address_bytes = P2PKHBitcoinAddress.from_bytes(public_key.to_string())
print(f'Private Key BAse58 Encoded: {base58.b58encode(private_key_raw.to_string()).decode("utf-8")}')
print(f'Private key not encoded: {private_key_raw.to_string().hex() + "01"}')
bitccoin_private_key = CBitcoinSecret.from_secret_bytes(private_key_raw.to_string())
bitccoin_private_key_bytes = bitccoin_private_key

# print(f"Bitcoin private key: {bitccoin_private_key_bytes.hex()[:-2]}")
print(f"Bitcoin private key: {bitccoin_private_key_bytes.hex()}")

# bitcoin_public_key_bytes = bitccoin_private_key_bytes.pub
print(f'Public key base58 Encoded: {base58.b58encode(private_key_raw.get_verifying_key().to_string())}')
print(f'bitcoin public not encoded: {private_key_raw.get_verifying_key().to_string()}')

def compress_public_key(public_key: bytes) -> bytes:
    x = public_key[:32]
    y = public_key[32:]
    if int.from_bytes(y, 'big') % 2 == 0:
        return b'\x02' + x
    else:
        return b'\x03' + x
    
bitcoin_public_key_bytes = compress_public_key(private_key_raw.get_verifying_key().to_string())
print(f"Bitcoin public key: {bitcoin_public_key_bytes.hex()}")

print(f"Bitcoin address from top: {bitcoin_address}")
bitcoin_address_bytes = P2PKHBitcoinAddress.from_pubkey(bitcoin_public_key_bytes)
print(f"Bitcoin address: {bitcoin_address_bytes}")

bitcoin_message_bytes = BitcoinMessage(message.decode())
print(f"Bitcoin message: {bitcoin_message_bytes.magic} {bitcoin_message_bytes.message}")

print(f'signature from top: {signature.hex()}')
# print(f'signature from top b64: {base64.b64encode(signature)}')
# signature_bytes = signature.__bytes__()
signature_bytes_b64 = base64.b64encode(bytes.fromhex(signature.hex())) 
print(f"Signature from top b64: {signature_bytes_b64}")

#sign message using bitcoin library
signature_bytes = SignMessage(bitccoin_private_key, bitcoin_message_bytes)
print(f"Signature bytes: {signature_bytes}")

print(f'signature_bytes_hex: {base64.b64decode(signature_bytes).hex()}')

# verify the message
print(f"Signature valid? {VerifyMessage(bitccoin_private_key, bitcoin_message_bytes, signature_bytes)}")

Private Key BAse58 Encoded: 4nUxdwXnJ6zcfmyptb57RVskAjUajKybpuSxyBAuCkjg
Private key not encoded: 3839cf91e611d1b8f743ad1aa3bf9a6673752e95c1e2fd4d07bca30cbc2f9bcf01
Bitcoin private key: 3839cf91e611d1b8f743ad1aa3bf9a6673752e95c1e2fd4d07bca30cbc2f9bcf01
Public key base58 Encoded: b'2kNMNRctgcX3QVcw1u5Yy27C4yxhYUJ3YDFT6eU64wHj4vpkwdv1Ceob18xHEdqtJugw26CAkBqfMRpGnBMnkSfW'
bitcoin public not encoded: b"Wj\x80qQ\xae{\xf4\xc2yV\nP\xf3\xed\xc2yo''xL\xf2P\xb3\x92!h\x8c\xf6\x19f\xe9\xe0\x8a\xb7\xd4\x95\xfe_\x07\x95\xd8G\x08\x80\x02b9{#\xac\xecE\x96\xa6\x1c\x02\x85\xc4\x8bs\x1eE"
Bitcoin public key: 03576a807151ae7bf4c279560a50f3edc2796f2727784cf250b39221688cf61966
Bitcoin address from top: 196fqHTSGuyHJYJH7MMs2GP49cd8ZHc1yG
Bitcoin address: 1NNSuRxt3i5k75Qz9TehrXnz1zfWDs2gdS
Bitcoin message: b'Bitcoin Signed Message:\n' b'\x18Bitcoin Signed Message:\nHello world!'
signature from top: b42cf72536589b6a8ed998f9a32d8a2cc7c77e24801b223a4368bfee8097b386350721d994c67bd0b1cbff8ed4f48e74cd19c974fb932976